In [3]:
import os
import re
import uuid
import sqlite3
from typing import List

from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma


In [4]:
# Load environment variables
load_dotenv()

True

In [5]:
# def is_file_registered(file_path, namespace: str) -> bool:
#     """
#     주어진 네임스페이스의 컬렉션이 벡터 데이터베이스에 이미 등록되어 있는지 확인합니다.
    
#     Args:
#         file_path (str): 벡터 데이터베이스 디렉토리 경로
#         namespace (str): 확인할 컬렉션의 네임스페이스
        
#     Returns:
#         bool: 해당 네임스페이스가 이미 등록되어 있으면 True, 아니면 False
#     """
#     path = os.path.join(file_path, namespace)
#     return os.path.isdir(path)

def is_file_registered(file_path, namespace: str) -> bool:
    """
    주어진 네임스페이스의 컬렉션이 벡터 데이터베이스에 이미 등록되어 있는지 확인합니다.
    
    Args:
        file_path (str): 벡터 데이터베이스 디렉토리 경로
        namespace (str): 확인할 컬렉션의 네임스페이스
        
    Returns:
        bool: 해당 네임스페이스가 이미 등록되어 있으면 True, 아니면 False
    """
    sqlite_path = os.path.join(file_path, "chroma.sqlite3")
    if not os.path.exists(sqlite_path):
        return False
    
    try:
        # SQLite 데이터베이스에 연결
        conn = sqlite3.connect(sqlite_path)
        cursor = conn.cursor()
        
        # collections 테이블에서 해당 네임스페이스 검색
        cursor.execute("SELECT COUNT(*) FROM collections WHERE name = ?", (namespace,))
        count = cursor.fetchone()[0]
        
        conn.close()
        return count > 0
    except sqlite3.Error:
        # SQLite 오류 발생 시 기존 방식으로 폴백
        path = os.path.join(file_path, namespace)
        return os.path.isdir(path)
    

def register_file(file_path, namespace: str):
    """
    새로운 컬렉션을 벡터 데이터베이스에 등록합니다.
    이 함수는 주어진 네임스페이스로 Chroma 인스턴스를 생성합니다.
    
    Args:
        file_path (str): 벡터 데이터베이스 디렉토리 경로
        namespace (str): 등록할 컬렉션의 네임스페이스
    """
    Chroma(persist_directory=file_path, collection_name=namespace)

In [6]:
def sanitize_namespace(file_path: str) -> str:
    """
    파일 경로에서 Chroma DB 컬렉션 네임스페이스로 사용할 수 있는 유효한 문자열을 생성합니다.
    
    Args:
        file_path (str): 원본 파일 경로
        
    Returns:
        tuple: (정제된 네임스페이스, 원본 파일명)
            - 정제된 네임스페이스는 영숫자, 점, 밑줄, 하이픈만 포함합니다.
            - 시작과 끝은 반드시 영숫자여야 합니다.
    """
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    # 허용 문자 외를 '_'로 치환
    ascii_namespace = re.sub(r"[^a-zA-Z0-9._-]", "_", base_name)
    # 시작/끝이 영숫자가 아닐 경우 'a'로 보정
    if not re.match(r"^[a-zA-Z0-9]", ascii_namespace):
        ascii_namespace = "a" + ascii_namespace
    if not re.match(r".*[a-zA-Z0-9]$", ascii_namespace):
        ascii_namespace = ascii_namespace + "a"
    return ascii_namespace, base_name

def load_pdf_chunks(file_path: str) -> List:
    """
    PDF 파일을 로드하고 작은 텍스트 청크로 분할합니다.
    
    Args:
        file_path (str): PDF 파일 경로
        
    Returns:
        List: Document 객체의 리스트. 각 Document는 텍스트 청크와 메타데이터를 포함합니다.
    """
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(docs)
    return chunks

def embed_and_upsert(database_path: str, chunks: List, namespace: str):
    """
    텍스트 청크를 임베딩하고 Chroma 벡터 데이터베이스에 업서트합니다.
    
    Args:
        database_path (str): 벡터 데이터베이스 저장 경로
        chunks (List): 임베딩할 Document 객체의 리스트
        namespace (str): 저장할 Chroma 컬렉션 네임스페이스
    """
    # 임베딩 모델 초기화
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Chroma 인스턴스 생성
    chroma = Chroma(
        persist_directory=database_path,
        collection_name=namespace,
        embedding_function=embeddings
    )

    # 문서 추가 (내부에서 텍스트, 메타데이터 및 벡터 자동 처리)
    print(f"📤 Adding {len(chunks)} documents to Chroma collection '{namespace}'...")
    chroma.add_documents(chunks)

    # Chroma DB에 저장
    print(f"✅ Upsert complete! namespace = '{namespace}'")

def main(file_path: str, database_path: str):
    """
    PDF 파일을 처리하여 벡터 데이터베이스에 저장하는 전체 프로세스를 관리합니다.
    
    Args:
        file_path (str): 처리할 PDF 파일 경로
        database_path (str): 벡터 데이터베이스 저장 경로
    """
    if not os.path.exists(file_path):
        print(f"❌ File does not exist: {file_path}")
        return

    # 네임스페이스를 파일명 기반 자동 생성
    namespace, basename = sanitize_namespace(file_path)

    if is_file_registered(file_path=database_path, namespace=namespace):
        print(f"ℹ️ Collection already exists: {basename} (namespace: {namespace})")
        return

    print(f"📄 Loading pdf: {file_path}")
    chunks = load_pdf_chunks(file_path)
    print(f"🔗 Number of chunks: {len(chunks)}")

    if not chunks:
        print("❌ No chunk is extracted from pdf.")
        return

    embed_and_upsert(database_path, chunks, namespace)

    # Register namespace (collection)
    register_file(file_path=database_path, namespace=namespace)
    print(f"✅ Registered collection: {basename} in Chroma.")

In [7]:
if __name__ == "__main__":
    # file_path = "data/2025학년도 전주대학교 정시 모집요강.pdf"
    file_path = "data/2024-1학기_일반대학원.pdf"
    
    database_path = 'database'
    main(file_path=file_path, database_path=database_path)

ℹ️ Collection already exists: 2024-1학기_일반대학원 (namespace: 2024-1________a)


In [12]:
def search_similar_documents(query: str, database_path: str, namespace: str, top_k: int = 3):
    """
    사용자 질의와 의미적으로 유사한 문서를 벡터 데이터베이스에서 검색합니다.
    
    Args:
        query (str): 사용자 질의
        database_path (str): 벡터 데이터베이스 경로
        namespace (str): 검색할 컬렉션 네임스페이스
        top_k (int, optional): 반환할 최대 문서 수. 기본값은 3입니다.
        
    Returns:
        List: 유사한 문서 목록
    """
    # 임베딩 모델 초기화
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    
    # Chroma 인스턴스 생성
    chroma = Chroma(
        persist_directory=database_path,
        collection_name=namespace,
        embedding_function=embeddings
    )
    
    # 유사 문서 검색
    results = chroma.similarity_search(query, k=top_k)
    
    return results

In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema.document import Document

def search_with_sklearn_similarity(query: str, database_path: str, namespace: str, top_k: int = 3):
    """
    사용자 질의와 의미적으로 유사한 문서를 검색하고 scikit-learn을 사용하여 코사인 유사도 점수를 계산합니다.
    
    Args:
        query (str): 사용자 질의
        database_path (str): 벡터 데이터베이스 경로
        namespace (str): 검색할 컬렉션 네임스페이스
        top_k (int, optional): 반환할 최대 문서 수. 기본값은 3입니다.
        
    Returns:
        List[Tuple]: (문서, 유사도 점수) 쌍의 리스트. 유사도 점수 내림차순으로 정렬됩니다.
    """
    # 임베딩 모델 초기화
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    
    # Chroma 인스턴스 생성
    chroma = Chroma(
        persist_directory=database_path,
        collection_name=namespace,
        embedding_function=embeddings
    )
    
    # 전체 문서 가져오기
    collection_data = chroma.get()
    
    if not collection_data['ids']:
        return []
    
    # 질의 임베딩 생성
    query_embedding = embeddings.embed_query(query)
    
    # 문서 임베딩과 질의 임베딩 간의 코사인 유사도 계산
    doc_embeddings = np.array(collection_data['embeddings'])
    query_embedding_array = np.array(query_embedding).reshape(1, -1)
    
    similarities = cosine_similarity(query_embedding_array, doc_embeddings)[0]
    
    # 문서, 메타데이터, 유사도 점수를 함께 저장
    results = []
    for i, (doc_id, doc_text, metadata, similarity) in enumerate(zip(
            collection_data['ids'],
            collection_data['documents'],
            collection_data['metadatas'],
            similarities
        )):
        doc = Document(page_content=doc_text, metadata=metadata)
        results.append((doc, similarity))
    
    # 유사도 점수로 내림차순 정렬
    results.sort(key=lambda x: x[1], reverse=True)
    
    # top-k 결과 반환
    return results[:top_k]

In [9]:
# scikit-learn을 사용한 코사인 유사도 검색 테스트
print("\n===== scikit-learn을 사용한 코사인 유사도 검색 =====\n")

queries = [
    "대학원 수업료에 대해 알려줘",
    "등록금 납부 방법은?",
    "학사일정에 대해 알려줘",
    "졸업 요건은 무엇인가요?"
]

for query in queries:
    print(f"\n===== 쿼리: {query} =====")
    results = search_with_sklearn_similarity(
        query=query, 
        database_path=database_path, 
        namespace=namespace, 
        top_k=3
    )
    
    for i, (doc, score) in enumerate(results):
        print(f"결과 {i+1}:")
        print(f"유사도 점수: {score:.4f}")  # 소수점 4자리까지 표시
        print(f"내용: {doc.page_content[:100]}...")  # 내용 일부만 출력
        print(f"메타데이터: {doc.metadata}\n")


===== scikit-learn을 사용한 코사인 유사도 검색 =====


===== 쿼리: 대학원 수업료에 대해 알려줘 =====


NameError: name 'namespace' is not defined